In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Load data
file1_path = '/aaw1313_data_s1.xlsx'  # Adjust if needed
file1_main_data = pd.read_excel(file1_path, sheet_name='population change by species NA')

# Dash app setup
app = JupyterDash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Endangered Species Visualization", style={'textAlign': 'center'}),

    # Dropdown to select category
    html.Div([
        html.Label("Select Category:"),
        dcc.Dropdown(
            id='category-dropdown',
            options=[
                {'label': 'Breeding Biome', 'value': 'Breeding.Biome'},
                {'label': 'Bird Group', 'value': 'bird.group'},
                {'label': 'Native Status', 'value': 'native'}
            ],
            value='Breeding.Biome'
        ),
    ], style={'width': '40%', 'margin': 'auto'}),

    # Textbox to input threshold for endangered
    html.Div([
        html.Label("Endangered Threshold (% Loss):"),
        dcc.Input(
            id='threshold-input',
            type='number',
            value=-50,  # Default threshold
            step=5,
        ),
    ], style={'width': '40%', 'margin': '20px auto'}),

    # Graph output
    dcc.Graph(id='endangered-graph'),
])

# Callback to update the graph
@app.callback(
    Output('endangered-graph', 'figure'),
    [Input('category-dropdown', 'value'),
     Input('threshold-input', 'value')]
)
def update_graph(category, threshold):
    # Recalculate endangered based on the user-specified threshold
    threshold_value = abs(threshold)  # Ensure threshold is positive for percentage loss
    filtered_data = file1_main_data.copy()
    filtered_data["Endangered"] = filtered_data["Loss_med"] <= -(threshold_value / 100) * filtered_data["last_year_popest"]

    # Group and calculate percentages
    summary = filtered_data.groupby(category)["Endangered"].mean() * 100
    summary = summary.reset_index()

    # Create bar plot
    fig = px.bar(
        summary,
        x=category,
        y="Endangered",
        title=f"Percentage of Endangered Species by {category}",
        labels={"Endangered": "Percentage Endangered", category: "Category"},
        color="Endangered",
    )
    return fig

# Run the app in Colab
app.run_server(mode='external')
